In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
import os
from datetime import datetime, date

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

chart_color = px.colors.qualitative.Plotly

In [2]:
if os.name =='nt':
    df_path = r'D:\Coding_pratice\_Data\GalaxyEducation\ICK'
else:
    df_path = '/Users/admin/_Work/Data/icanKID'

# df_User = pd.read_csv(
#     os.path.join(df_path,'df_User.csv')
#     )
df_User = pd.read_csv(
    os.path.join(df_path,'df_User_Child_DeviceBrand.csv')
    )
df_Engagement = pd.read_csv(
    os.path.join(df_path, 'df_Engagement.csv')
    )
df_Engagement_updated = pd.read_csv(
os.path.join(df_path, 'df_Engagement_updated.csv')
    )
df_Payment = pd.read_csv(
    os.path.join(df_path, 'df_Payment.csv')
    )
df_fake_user = pd.read_csv(
    os.path.join(df_path, 'df_fake_paid_users.csv')
    )
# df_DeviceBrand = pd.read_csv('/Users/admin/_Work/Data/icanKID/df_DeviceBrand.csv')

In [3]:
# Discard fake user
df_User = df_User[~df_User['UserID'].isin(df_fake_user['UserID'])]
df_Engagement = df_Engagement[~df_Engagement['UserID'].isin(df_fake_user['UserID'])]
df_Payment = df_Payment[~df_Payment['UserID'].isin(df_fake_user['UserID'])]

In [4]:
# Get user's Age
df_User['monthBday'] = pd.to_datetime(df_User.ChildBday).dt.month
df_User['Age'] = (pd.to_datetime(date.today()) - pd.to_datetime(df_User.ChildBday)).astype('<m8[Y]').astype('int')

In [5]:
def day_of_week_func(x):
    """
        Input: Normal date

        Return: Day in week
    """
    # Why don't need .dt if apply to Series
    # https://stackoverflow.com/questions/62803633/timestamp-object-has-no-attribute-dt#_=_
    x = pd.to_datetime(x).dayofweek
    day_of_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    dict_dayofweek = {i:v for i,v in enumerate(day_of_week)}

    return dict_dayofweek[x]

def slope_function(x, y):
    # x: Timeseries
    # y: Datapoint
    n = len(x)
    x_sum = np.sum(x)
    y_sum = np.sum(y)
    xy_sum = np.sum(x*y)
    x2_sum = np.sum(x**2)

    denominator = n * x2_sum - x_sum**2
    if denominator == 0:
        return 0
    
    else:
        return (n * xy_sum - x_sum * y_sum) / (n * x2_sum - x_sum**2)

def slope_function_2(x, y):
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    sx = np.std(x, ddof=1)
    sy = np.std(y, ddof=1)
    denominator = (len(x) * sx * sy)
    if denominator == 0:
        return 0
    else:
        r = np.sum((x - x_mean) * (y - y_mean)) / (len(x) * sx * sy)
        m = r * (sy / sx)
        return m

def accumulate_func(Series_values:pd.Series, take_abs:int):
    if take_abs:
        accumulate = lambda x,y: np.abs(x-y)
    else:
        accumulate = lambda x,y: x-y

    return ([
        accumulate(Series_values.iloc[i], Series_values.iloc[i-1]) for i in range(1, len(Series_values))
        ] if len(Series_values) > 1 else [0])

def interest_rate(Engagement, userID):
    usedDay_column = 'usedDays'
    if usedDay_column in Engagement.columns:
        pass
    else:
        usedDay_column = 'nth_day_from_registration'

    temp = Engagement[Engagement.UserID == userID].sort_values(by=usedDay_column)
    UserID_df = temp[['UserID']].head(1).reset_index(drop=True)
    temp['interest'] = temp['UsageTime'] * temp['Records']

    temp_check = temp.groupby('ActionType')['interest'].agg(
        total_interest=lambda x: sum(x),
        average_interest=lambda x: np.mean(x),
        # accumulate_inc_avg=lambda x: np.mean([
        #     x.iloc[i] - x.iloc[i-1] if (i != 0) and (x.iloc[i] - x.iloc[i-1]) > 0 else 0 for i in range(1, len(x))]),
        # accumulate_dec=lambda x: sum([
        #     x.iloc[i] - x.iloc[i-1] if (i != 0) and (x.iloc[i] - x.iloc[i-1]) < 0 else 0 for i in range(len(x))]),
        # total_accumulate_avg=lambda x: np.mean(accumulate_func(x, True)),
        accumulate=lambda x: np.sum(accumulate_func(x, False)),

    ).reset_index()

    grouped = temp.groupby('ActionType')
    # temp_check_2 = grouped.agg(
    #     Slope=pd.NamedAgg(
    #         column='interest', 
    #         aggfunc=lambda x: slope_function(
    #             grouped[usedDay_column].get_group(x.name),
    #             x       
    # ))).reset_index()

    temp_check_3 = grouped.agg(
        Slope_2=pd.NamedAgg(
            column='interest', 
            aggfunc=lambda x: slope_function_2(
                grouped[usedDay_column].get_group(x.name),
                x       
    ))).reset_index()

    # temp_check = pd.merge(temp_check, temp_check_2, on='ActionType')
    temp_check = pd.merge(temp_check, temp_check_3, on='ActionType')

    # To tell if the recent activity increase or decrease throughout the span of 't' days
    # Value range between [-1,1], with 0 as no trend (only 1 day of active)
    temp_check.reset_index(inplace=True, drop=True)
    temp_check['Slope_norm'] = temp_check['Slope_2'] / temp_check['average_interest']
    # display(temp_check)

    temp_check.drop(
        columns=['Slope_2'], 
        inplace=True)

    most_active = temp_check.iloc[[temp_check['total_interest'].idxmax()]].reset_index(drop=True)
    most_accumulate = temp_check.iloc[[temp_check['accumulate'].idxmax()]].reset_index(drop=True)
    
    most_active.fillna(0, inplace=True)
    most_active.drop(columns=['accumulate','average_interest'], inplace=True)
    most_accumulate.fillna(0, inplace=True)
    most_accumulate.drop(columns=['total_interest', 'average_interest'], inplace=True)

    most_active.rename(columns={
        "ActionType": "MostInterest",
        "total_interest": "TotalInterestRatio",
        "Slope_norm": "InterestTrend"
    },inplace=True)

    most_accumulate.rename(columns={
        "ActionType": "MostAccumulate",
        "accumulate": "TotalAccuRatio",
        "Slope_norm": "AccuTrend"
    }, inplace=True)
    total_interest = temp_check['total_interest'].sum()
    total_accumulate = np.sum(np.abs(temp_check['accumulate']))

    most_active['TotalInterestRatio'] = most_active['TotalInterestRatio'] / total_interest
    if total_accumulate == 0:
        most_accumulate['TotalAccuRatio'] = 0
    else:
        most_accumulate['TotalAccuRatio'] = most_accumulate['TotalAccuRatio'] / total_accumulate
   
    return pd.concat([UserID_df, most_active, most_accumulate], axis=1)

def getEvent_before_cutoff(df_events, cutOffDay=3):
    days_from_regis = df_events.groupby('UserID')[['nth_day_from_registration']].min().reset_index()
    days_from_regis.rename(
        columns={
            "nth_day_from_registration": "MinFirstDay"
        },inplace=True
    )
    days_from_regis['cutoffDay'] = days_from_regis.MinFirstDay + cutOffDay
    days_from_regis['lateUser'] = (days_from_regis['MinFirstDay'] > 0).astype(int)

    Engagement = df_events.merge(days_from_regis, on='UserID',how='left')
    Engagement = Engagement[
        (Engagement['cutoffDay'] - Engagement['nth_day_from_registration']) > 0
        ].reset_index(drop=True)
    Engagement['usedDays'] = Engagement['nth_day_from_registration'] - Engagement['MinFirstDay']
    return Engagement

def setLabel(df_events, df_Payment, markDay=30):
    df_events = df_events.merge(df_Payment[['UserID','gapday']], how='left', on='UserID')
    """
        If label = True -> User paid before markDay
        else -> Free user
    """
    df_events['Label'] = ((df_events['gapday'] - df_events['MinFirstDay']) <= markDay).astype(int)

    return df_events

def create_dataframe(dataframes):
    df_User, df_Engagement, df_Payment = dataframes

    # Rebrand rarely used brand
    rep_brand = df_User['DeviceBrand'].value_counts()[
    df_User['DeviceBrand'].value_counts() < df_User['DeviceBrand'].value_counts().mean()].index.values
    df_User['DeviceBrand'] = df_User['DeviceBrand'].apply(lambda x: 'Other brand' if x in rep_brand else x)

    device_brand_df = pd.get_dummies(
            df_User[['UserID', 'DeviceBrand']],
            columns=['DeviceBrand'], prefix= '', prefix_sep=''
        )

    # Get total child
    print("Counting total children")
    total_child = df_User.groupby(by='UserID')[['ChildID']].nunique().reset_index()
    total_child.rename(
        columns={
            'ChildID':'TotalChild'
        }, inplace=True
    )

    # Get number of female and male children for each user
    print("Counting number of boy and girl seperate")
    no_fe_n_ma = df_User.pivot_table(index='UserID', columns='ChildGender', values='ChildID', aggfunc='count').reset_index()
    no_fe_n_ma.rename(
        columns={
            'female':'No.Female',
            'male': 'No.Male'
        }, inplace=True
    )
    no_fe_n_ma.fillna(0, inplace=True)

    # Get median age children
    print("Get median children age")
    age_child = df_User.groupby(by='UserID')[['Age']].median().reset_index()
    age_child.rename(
        columns={
            'Age':'MedianChildAge'
        }
    )

    # Rename certain columns to match with general database
    # df_Engagement['lastuse_byhour'] = df_Engagement['lastuse'].apply(lambda x: pd.to_datetime(x).strftime("%H:%M:%S"))
    df_Engagement.rename(
        columns={
            'open_time': 'Opentime',
            'payment_screen_time': 'PaymentScreenTime'
        }, inplace=True
    )

    # Acquire dataframe Engagement by condition
    # df_Engagement = df_Engagement[df_Engagement.nth_day_from_registration <=7]
    # df_Engagement = df_Engagement[condition]
    
    """
        Idea fromm: https://stackoverflow.com/questions/47360510/pandas-groupby-and-aggregation-output-should-include-all-the-original-columns-i
    """
    print("Obtain Users event base on condition")
    df_merge = df_Engagement.groupby(by=['UserID', 'EventDate', 'ActionType'], as_index=False).progress_apply(
        lambda x : x.sum() if ((x.dtypes=='int64') | (x.dtypes=='float64')).any() else x.head(1)
        )
    print('df_merge: \n',df_merge)
    # Retain labels and important features

    labels_df = df_merge[['UserID', 'cutoffDay', 'lateUser', 'Label']]

    # Create base df for merge
    print("Create base column for merge")
    based_df = df_Engagement.groupby('UserID', as_index=False).progress_apply(
        lambda x: x.head(1))[['UserID', 'UserJoinedDate']].reset_index(drop=True)
    print("Unique user: ", based_df.shape[0])

    # Total of Action/Events
    print("Get total Open time/Usage Time/Record/Payment screen time")
    total_merge = df_merge.groupby(by='UserID')[['Opentime', 'UsageTime', 'Records', 'PaymentScreenTime']].progress_apply(sum).reset_index()
    total_merge.rename(
        columns={'Opentime':'TotalOpentime', 'UsageTime':'TotalUsageTime', 
                'Records':'TotalRecords', 'PaymentScreenTime': 'TotalPayscreentime'}, inplace=True
    )

    # Sum activity merge
    print('Get Open time/Usage Time/Record/Payment screen time base on Action type')
    activity_merge = pd.pivot_table(df_merge, 
                                index='UserID', 
                                columns='ActionType', 
                                values=['Opentime', 'UsageTime', 'Records','PaymentScreenTime'],
                                aggfunc=np.sum)

    activity_merge.columns = ["".join((i,j.title())) for i,j in activity_merge.columns]
    activity_merge.reset_index(inplace=True)
    activity_merge.fillna(0, inplace=True)

    print("Get interest...")
    # map_func = np.vectorize(lambda x: interest_rate(df_Engagement, x))
    # interest_df = map_func(based_df['UserID'])
    interest_df = [interest_rate(df_Engagement, n) for n in tqdm(based_df['UserID'])]
    interest_df = pd.concat(interest_df, axis=0, ignore_index=True)
    
    print(interest_df)
    merge_dfs = [
        total_child, no_fe_n_ma, age_child, total_merge, activity_merge, interest_df, device_brand_df, labels_df
        ]
    
    print("Star merging....")
    for df in merge_dfs:
        df.reset_index(inplace=True, drop=True)
        based_df = based_df.merge(df, on='UserID', how='left')
        based_df.drop_duplicates(subset=['UserID'], keep='first', inplace=True, ignore_index=True)

    print("Total user with compress events: ", based_df.shape)
    return based_df

In [7]:
Engagements = getEvent_before_cutoff(df_Engagement.iloc[:100], cutOffDay=3)
Engagements = setLabel(Engagements, df_Payment, markDay=30)
list_dataframes =[
    df_User,
    Engagements,
    df_Payment
]
condition_df = create_dataframe(list_dataframes)
condition_df

Counting total children
Counting number of boy and girl seperate
Get median children age
Obtain Users event base on condition


100%|██████████| 100/100 [00:00<00:00, 496.22it/s]


df_merge: 
      EventDate                                UserID     ActionType  \
0   2022-07-12  040b1165-f80b-4c2b-8608-af46735e794b  entertainment   
1   2022-07-14  0431ddef-cd95-4e95-8c00-8bc4bed18651          learn   
2   2022-06-09  05150eaf-4b68-40a0-9893-2a2786bd907e      discovery   
3   2023-01-08  0587b7e7-fe50-4e34-92a1-122fd587599b  entertainment   
4   2022-10-01  08cb10c2-5970-44ce-9c71-622e868ab485          learn   
..         ...                                   ...            ...   
95  2023-01-18  f62af61c-2b9e-46b0-b950-c5277306a693          learn   
96  2023-02-28  f7735c76-18f5-4a1d-93ca-cddff65f6b89          learn   
97  2022-06-05  f961864b-6fa1-48b1-9646-690014a2e7c6          learn   
98  2022-07-09  fa4ee85b-4040-4fc7-9397-ca28e01fdcd9          learn   
99  2022-09-05  fd7c7b47-b8a1-47fe-b2ab-e6fa3fe8631f          learn   

                      UserJoinedDate  nth_day_from_registration  Opentime  \
0   2022-06-26 07:03:27.635000+07:00                      

100%|██████████| 100/100 [00:00<00:00, 1917.00it/s]


Unique user:  100
Get total Open time/Usage Time/Record/Payment screen time


100%|██████████| 100/100 [00:00<00:00, 619.95it/s]


Get Open time/Usage Time/Record/Payment screen time base on Action type
Get interest...


100%|██████████| 100/100 [00:01<00:00, 62.55it/s]


                                  UserID   MostInterest  TotalInterestRatio  \
0   c65f5b3f-e220-4c2a-a94e-74c4749f0783          learn                 1.0   
1   66fd7717-ee82-4f85-8702-3111f0179d4d          learn                 1.0   
2   c19e1a88-5ff6-464d-9bf3-62d8f3ba806c  entertainment                 1.0   
3   fd7c7b47-b8a1-47fe-b2ab-e6fa3fe8631f          learn                 1.0   
4   98e54195-30a2-4cb3-b286-2bed25b10ba0      discovery                 1.0   
..                                   ...            ...                 ...   
95  3dfeb07a-911f-411e-b519-0180f3069895          learn                 1.0   
96  a264d35d-5ef1-46a7-b2c8-56f9a38a6fc7  entertainment                 1.0   
97  d931fe64-ea79-4f3b-b37a-840fe2d05427          learn                 1.0   
98  35aa2bba-9dc6-4bb1-a31c-559be4408ea6  entertainment                 1.0   
99  d05d62e6-16e5-4e6c-8ed3-92dd12e9f35e  entertainment                 1.0   

    InterestTrend MostAccumulate  TotalAccuRatio  A

,UserID,UserJoinedDate,TotalChild,No.Female,No.Male,Age,TotalOpentime,TotalUsageTime,TotalRecords,TotalPayscreentime,OpentimeDiscovery,OpentimeEntertainment,OpentimeLearn,PaymentScreenTimeDiscovery,PaymentScreenTimeEntertainment,PaymentScreenTimeLearn,RecordsDiscovery,RecordsEntertainment,RecordsLearn,UsageTimeDiscovery,UsageTimeEntertainment,UsageTimeLearn,MostInterest,TotalInterestRatio,InterestTrend,MostAccumulate,TotalAccuRatio,AccuTrend,Other brand,apple,oppo,samsung,cutoffDay,lateUser,Label
0,c65f5b3f-e220-4c2a-a94e-74c4749f0783,2022-05-28 09:11:16.181000+07:00,1,0.0,2.0,5.0,1.0,33.80,26.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,26.0,0.00,0.00,33.80,learn,1.0,0.0,learn,0,0.0,0,0,0,1,5,1,1
1,66fd7717-ee82-4f85-8702-3111f0179d4d,2023-01-16 20:51:29.481000+07:00,1,0.0,1.0,4.0,2.0,5.75,8.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,8.0,0.00,0.00,5.75,learn,1.0,0.0,learn,0,0.0,1,0,0,0,23,1,0
2,c19e1a88-5ff6-464d-9bf3-62d8f3ba806c,2022-07-04 17:02:56.496000+07:00,2,2.0,2.0,5.5,1.0,9.57,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.00,9.57,0.00,entertainment,1.0,0.0,entertainment,0,0.0,0,0,0,1,150,1,1
3,fd7c7b47-b8a1-47fe-b2ab-e6fa3fe8631f,2022-08-31 21:56:29.099000+07:00,1,0.0,2.0,6.0,2.0,6.40,8.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,8.0,0.00,0.00,6.40,learn,1.0,0.0,learn,0,0.0,0,1,0,0,8,1,1
4,98e54195-30a2-4cb3-b286-2bed25b10ba0,2022-07-11 17:24:30.188000+07:00,1,1.0,0.0,3.0,1.0,4.58,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.58,0.00,0.00,discovery,1.0,0.0,discovery,0,0.0,0,1,0,0,53,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3dfeb07a-911f-411e-b519-0180f3069895,2022-05-03 21:22:50.628000+07:00,1,1.0,0.0,8.0,0.0,5.58,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.00,0.00,5.58,learn,1.0,0.0,learn,0,0.0,0,1,0,0,3,0,0
96,a264d35d-5ef1-46a7-b2c8-56f9a38a6fc7,2023-01-31 13:44:36.745000+07:00,1,2.0,0.0,5.0,1.0,35.57,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.00,35.57,0.00,entertainment,1.0,0.0,entertainment,0,0.0,0,0,0,1,79,1,1
97,d931fe64-ea79-4f3b-b37a-840fe2d05427,2023-04-12 10:08:24.062000+07:00,1,0.0,1.0,2.0,1.0,1.20,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.00,0.00,1.20,learn,1.0,0.0,learn,0,0.0,0,1,0,0,10,1,0
98,35aa2bba-9dc6-4bb1-a31c-559be4408ea6,2022-06-22 16:32:31.975000+07:00,1,0.0,1.0,4.0,3.0,12.45,7.0,3.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,7.0,0.0,0.00,12.45,0.00,entertainment,1.0,0.0,entertainment,0,0.0,0,0,0,1,32,1,1


In [45]:
# 00015787-1ed9-45b9-9c5f-2613e2ca8409
# 00125ee4-9a93-465e-8e7a-556d1d6ca10e
# fffb6dfa-af6a-4e9e-93a7-4851632e43cd
# 0004052d-a54c-49cf-bb66-7747371304e8

# sample_ID = Engagements[Engagements['Label']==1].UserID.sample(1).values[0]
sample_ID = '00002520-3fe9-4d4b-9e1f-e4a72386bf47'

display(sample_ID)
temp = Engagements[Engagements.UserID ==sample_ID].sort_values(by='nth_day_from_registration')
temp['interest'] = temp['UsageTime'] * temp['Records']

fig = px.line(
    temp, 
    x='nth_day_from_registration',
    y='interest',
    color='ActionType',
    markers=True
)
fig.show()

'00002520-3fe9-4d4b-9e1f-e4a72386bf47'

In [48]:
start_time=datetime.now()
temp_check = interest_rate(Engagements, sample_ID)
display((datetime.now()- start_time).total_seconds())
temp_check

0.072276

,UserID,MostInterest,TotalInterestRatio,InterestTrend,MostAccumulate,TotalAccuRatio,AccuTrend
0,00002520-3fe9-4d4b-9e1f-e4a72386bf47,learn,1.0,0.0,learn,NaN,0.0
